# Introduction 

Consider I'm a part of a crisis response team, and I want to identify how hospitals have been responding to crash collisions in New York City.

<center>
<img src="intro_image.jpeg" width="450"><br/>
</center>

In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

I'll use the `embed_map()` function to visualize our maps.

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## Visualizing the collision data.

Loading NYC 'collisions' data tracking motor vehicle collisions in 2013-2018


In [3]:
collisions = gpd.read_file("NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,...,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,...,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


In [4]:
len(collisions)

261905

Using the "LATITUDE" and "LONGITUDE" columns to create an interactive map to visualize the collision data.

In [5]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Visualize the collision data
mc = MarkerCluster()
for idx, row in collisions.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']]))
m_1.add_child(mc)

# Show the map
embed_map(m_1, "q_1.html")

![Q1](q1.jpg)

### Understanding hospital coverage.

 Loading NYC hospitals data

In [6]:
hospitals = gpd.read_file("nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


In [7]:
len(hospitals)

62

Using the "latitude" and "longitude" columns to visualize the hospital locations. 

In [8]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Visualize the hospital locations
for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']]).add_to(m_2)
        
# Show the map
embed_map(m_2, "q_2.html")

![Q1](q2.jpg)

### When was the closest hospital more than 10 kilometers away?

To answer the above question, I will create a DataFrame `outside_range` containing all rows from `collisions` with crashes that occurred more than 10 kilometers from the closest hospital.

Note that both `hospitals` and `collisions` have EPSG 2263 as the coordinate reference system, and EPSG 2263 has units of meters.

In [9]:
# using buffer and unary_union from shapely library to create a union of all hospitals outside of 10km radius of the collision
coverage = hospitals.geometry.buffer(10000) # 10km
my_union = coverage.geometry.unary_union 

outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: my_union.contains(x))]

Calculating the `percentage of collisions` that occurred more than 10 kilometers away from the closest hospital.

In [10]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.12%


## Making a recommender.

When collisions occur in distant locations, it becomes even more vital that injured persons are transported to the nearest available hospital.

With this in mind, I decided to create a recommender that:
- takes the location of the crash (in EPSG 2263) as input,
- finds the closest hospital (where distance calculations are done in EPSG 2263), and 
- returns the name of the closest hospital. 

In [11]:
# function to find the best hospital that is closest to the collision 
def best_hospital(collision_location):
    distances = hospitals.geometry.distance(collision_location)
    name = hospitals.iloc[distances.idxmin()]['name']
    return name

# Testing the function best_hospital
print(best_hospital(outside_range.geometry.iloc[0]))

CALVARY HOSPITAL INC


### Which hospital is under the highest demand?

Considering only collisions in the `outside_range` DataFrame, which hospital has the highest demand? 

In [12]:
# find the hospital with highest demand
names = outside_range.geometry.apply(best_hospital)
highest_demand = names.value_counts().idxmax()

In [13]:
highest_demand

'JAMAICA HOSPITAL MEDICAL CENTER'

### Where should the city construct new hospitals?

Visualizing the hospital locations, in addition to collisions that occurred more than 10 kilometers away from the closest hospital. 

In [14]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=11) 

coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000) # 10km coveraeg buffer
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6) # GeoJson for coverage of hospitals
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6) # Heatmap for outside range
folium.LatLngPopup().add_to(m_6) # Popup for latitude and longitude to recommend new hospitals

embed_map(m_6, 'm_6.html')

![Q1](m6.jpg)

**I used the above map to click on the map to see a pop-up with the corresponding location in latitude and longitude, where the heatmap shows the higher density of crashes that occurred more than 10 kilometers away from the closest hospital.**

**Let's say that the city wants to build new hospitals. They want to have a goal of bringing the percentage of collisions that occurred more than 10 kilometers away from the closest hospital to less than ten percent without worrying about what potential buildings would have to be removed in order to build the hospitals. I tried getting it down to 10 percent with recommending only one new hospital, but it was not enough. Hence I decided to recommend **two new hospitals** at two different locations. This brought down the percentage to less than ten percent - **`9.12 percent`**.**

##### I developed the following code with the help of some online resources that takes in the proposed location of two new hospitals and returns the percentage of collisions that occurred more than 10 kilometers away from the closest hospital along with the map showing the proposed locations and their coverage. This code helped me to experiment with different locations without worrying about writing the same code over and over again.

In [15]:
# proposed location of hospital 1
lat_1 = 40.6714
long_1 = -73.8492

# proposed location of hospital 2
lat_2 = 40.6702
long_2 = -73.7612


# function to check if new hospital locations will get the percentage of collisions more than 10km to <10%
def new_hospital(lat_1, long_1, lat_2, long_2):
    new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
    new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
    new_gdf.crs = {'init' :'epsg:4326'}
    new_gdf = new_gdf.to_crs(epsg=2263)
    # get new percentage
    new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
    new_my_union = new_coverage.geometry.unary_union
    new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
    new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
    print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
    # make the map
    m = folium.Map(location=[40.7, -74], zoom_start=11) 
    folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
    folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
    for idx, row in new_gdf.iterrows():
        Marker([row['Latitude'], row['Longitude']]).add_to(m)
    HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
    folium.LatLngPopup().add_to(m)
    display(embed_map(m, 'q_6.html'))

# Testing the function new_hospital
new_hospital(lat_1, long_1, lat_2, long_2)

/Users/admin/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


(NEW) Percentage of collisions more than 10 km away from the closest hospital: 9.12%


![Q1](q6.jpg)

### Proposed Location of New Hospitals:

**`40.6714, -73.8492`** and **`40.6702, -73.7612`**

### By strategically placing two new hospitals at these coordinates, we can significantly enhance emergency response times. This proactive approach aims to reduce the percentage of collisions occurring outside the 10 km coverage of hospitals from 15.12% to a more effective 9.12%. This data-driven analysis, spanning from 2013 to 2018, underscores the potential to save lives by swiftly reaching accident sites.